In [1]:
from allennlp.commands.evaluate import *
from kb.include_all import *
from allennlp.nn import util as nn_util
from allennlp.common.tqdm import Tqdm
import torch
import warnings
warnings.filterwarnings("ignore")
archive_file = "knowbert_wiki_wordnet_model"
cuda_device = -1
# line = "banana\tcolor\tyellow"

/nas/home/gujiashe/miniconda3/envs/knowbert/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/nas/home/gujiashe/miniconda3/envs/knowbert/lib/python3.6/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:51: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  UserWarning)


In [2]:
import logging

logger = logging.getLogger()  # 不加名称设置root logger
logger.setLevel(logging.INFO)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s: - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')

# 使用FileHandler输出到文件
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)

# 使用StreamHandler输出到屏幕
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

# 添加两个Handler
logger.addHandler(ch)
logger.addHandler(fh)

In [3]:
archive = load_archive(archive_file, cuda_device)
config = archive.config
prepare_environment(config)

2022-02-22 17:25:51 - allennlp.models.archival - INFO: - loading archive file knowbert_wiki_wordnet_model
2022-02-22 17:25:51 - allennlp.common.params - INFO: - vocabulary.type = default
2022-02-22 17:25:51 - allennlp.data.vocabulary - INFO: - Loading token dictionary from knowbert_wiki_wordnet_model/vocabulary.
2022-02-22 17:25:52 - allennlp.common.from_params - INFO: - instantiating class <class 'allennlp.models.model.Model'> from params {'bert_model_name': 'bert-base-uncased', 'soldered_kgs': {'wiki': {'entity_linker': {'contextual_embedding_dim': 768, 'entity_embedding': {'embedding_dim': 300, 'sparse': False, 'trainable': False, 'vocab_namespace': 'entity_wiki'}, 'namespace': 'entity_wiki', 'span_encoder_config': {'hidden_size': 300, 'intermediate_size': 1024, 'num_attention_heads': 4, 'num_hidden_layers': 1}, 'type': 'entity_linking_with_candidate_mentions'}, 'should_init_kg_to_bert_inverse': False, 'span_attention_config': {'hidden_size': 300, 'intermediate_size': 1024, 'num_att

In [4]:
# config = Params.from_file("/home/scruple/kb/knowbert_wiki_wordnet_model/config.json")

In [5]:
reader_params = config.pop('dataset_reader')
# print(reader_params.as_dict())
if reader_params['type'] == 'multitask_reader':
    reader_params = reader_params['dataset_readers']['language_modeling']
# reader_params['num_workers'] = 0
validation_reader_params = {
    "type": "food",
    "tokenizer_and_candidate_generator": reader_params['base_reader']['tokenizer_and_candidate_generator'].as_dict()
}
dataset_reader = DatasetReader.from_params(Params(validation_reader_params))
vocab = dataset_reader._tokenizer_and_candidate_generator.bert_tokenizer.vocab
token2word = {}
for k, v in vocab.items():
    token2word[v] = k

2022-02-09 23:06:47 - allennlp.common.params - INFO: - Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
2022-02-09 23:06:47 - allennlp.common.params - INFO: - CURRENTLY DEFINED PARAMETERS: 
2022-02-09 23:06:47 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.bert_model_type = bert-base-uncased
2022-02-09 23:06:47 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.do_lower_case = True
2022-02-09 23:06:47 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.entity_candidate_generators.wiki.type = wiki
2022-02-09 23:06:47 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.entity_candidate_generators.word

In [7]:
instances = dataset_reader.read("../data/food_compact.tsv")

201it [00:00, 3092.85it/s]


In [8]:
print(instances[69]["tokens"])

TextField of length 8 with text: 
 		[[CLS], constituent, part, of, egg, is, [MASK], [SEP]]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'}


In [9]:
model = archive.model
model.eval()


print("start")
# metrics = evaluate(model, instances, iterator, cuda_device, "")
data_iterator = DataIterator.from_params(Params(
    {"type": "basic", "batch_size": 1}
))
data_iterator.index_with(model.vocab)
iterator = data_iterator(instances,
                            num_epochs=1,
                            shuffle=False)
logger.info("Iterating over dataset")
generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))
with open('prompt2.txt', 'wt') as f:
    for instance in generator_tqdm:
        batch = nn_util.move_to_device(instance, cuda_device)
        output_dict = model(**batch)
        pooled_output = output_dict.get("pooled_output")
        contextual_embeddings = output_dict.get("contextual_embeddings")
        prediction_scores, seq_relationship_score = model.pretraining_heads(
        contextual_embeddings, pooled_output
        )
        prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])
        # print("================")
        # print(prediction_scores)
        # print("================")
        idx = torch.argmax(prediction_scores, dim = 1)
        line = ""
        for i in idx.cpu().numpy():
            line += token2word[i] + " "
        
        print(line, file=f)

    

2022-02-09 23:07:59 - allennlp.common.from_params - INFO: - instantiating class <class 'allennlp.data.iterators.data_iterator.DataIterator'> from params {'type': 'basic', 'batch_size': 1} and extras set()
2022-02-09 23:07:59 - allennlp.common.params - INFO: - type = basic
2022-02-09 23:07:59 - allennlp.common.from_params - INFO: - instantiating class <class 'allennlp.data.iterators.basic_iterator.BasicIterator'> from params {'batch_size': 1} and extras set()
2022-02-09 23:07:59 - allennlp.common.params - INFO: - batch_size = 1
2022-02-09 23:07:59 - allennlp.common.params - INFO: - instances_per_epoch = None
2022-02-09 23:07:59 - allennlp.common.params - INFO: - max_instances_in_memory = None
2022-02-09 23:07:59 - allennlp.common.params - INFO: - cache_instances = False
2022-02-09 23:07:59 - allennlp.common.params - INFO: - track_epoch = False
2022-02-09 23:07:59 - allennlp.common.params - INFO: - maximum_samples_per_batch = None
2022-02-09 23:07:59 - root - INFO: - Iterating over datas

start


100%|██████████| 201/201 [01:08<00:00,  2.95it/s]


In [6]:
# config = Params.from_file("/home/scruple/kb/knowbert_wiki_wordnet_model/config.json")

In [4]:
reader_params = config.pop('dataset_reader')
if reader_params['type'] == 'multitask_reader':
    reader_params = reader_params['dataset_readers']['language_modeling']
# reader_params['num_workers'] = 0
validation_reader_params = {
    "type": "kg_probe",
    "tokenizer_and_candidate_generator": reader_params['base_reader']['tokenizer_and_candidate_generator'].as_dict()
}
dataset_reader = DatasetReader.from_params(Params(validation_reader_params))

vocab = dataset_reader._tokenizer_and_candidate_generator.bert_tokenizer.vocab
token2word = {}
for k, v in vocab.items():
    token2word[v] = k

2022-02-22 17:26:14 - allennlp.common.params - INFO: - Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
2022-02-22 17:26:14 - allennlp.common.params - INFO: - CURRENTLY DEFINED PARAMETERS: 
2022-02-22 17:26:14 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.bert_model_type = bert-base-uncased
2022-02-22 17:26:14 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.do_lower_case = True
2022-02-22 17:26:14 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.entity_candidate_generators.wiki.type = wiki
2022-02-22 17:26:14 - allennlp.common.params - INFO: - dataset_reader.dataset_readers.language_modeling.base_reader.tokenizer_and_candidate_generator.entity_candidate_generators.word

duplicate_mentions_cnt:  6777
end of p_e_m reading. wall time: 1.2795546372731528  minutes
p_e_m_errors:  0
incompatible_ent_ids:  0


2022-02-22 17:28:15 - allennlp.common.from_params - INFO: - instantiating class <class 'kb.common.MentionGenerator'> from params {'entity_file': 'https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wordnet/entities.jsonl', 'type': 'wordnet_mention_generator'} and extras set()
2022-02-22 17:28:15 - allennlp.common.params - INFO: - tokenizer_and_candidate_generator.entity_candidate_generators.wordnet.type = wordnet_mention_generator
2022-02-22 17:28:15 - allennlp.common.from_params - INFO: - instantiating class <class 'kb.wordnet.WordNetCandidateMentionGenerator'> from params {'entity_file': 'https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wordnet/entities.jsonl'} and extras set()
2022-02-22 17:28:15 - allennlp.common.params - INFO: - tokenizer_and_candidate_generator.entity_candidate_generators.wordnet.entity_file = https://allennlp.s3-us-west-2.amazonaws.com/knowbert/wordnet/entities.jsonl
2022-02-22 17:28:15 - allennlp.common.params - INFO: - tokenizer_and_candidate_generator.en

In [5]:
instances = dataset_reader.read("/nas/home/gujiashe/kb/sentences_birth.tsv")
print(instances[0])

11166it [00:17, 649.29it/s]

Instance with fields:
 	 tokens: TextField of length 20 with text: 
 		[[CLS], [MASK], [MASK], was, born, on, 24, april, 1934, ., his, mother, is, lakshmi, raja, ##go,
		##pala, ##cha, ##ri, [SEP]]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'} 
 	 segment_ids: ArrayField with shape: (20,). 
 	 candidates:  
 	 lm_label_ids: TextField of length 20 with text: 
 		[[CLS], tara, gandhi, was, born, on, 24, april, 1934, ., his, mother, is, lakshmi, raja, ##go,
		##pala, ##cha, ##ri, [SEP]]
 		and TokenIndexers : {'lm_labels': 'SingleIdTokenIndexer'} 
 	 mask_indicator: ArrayField with shape: (20,). 



In [12]:
import pandas as pd
 
birth_df = pd.read_csv('/nas/home/gujiashe/kb/sentences_birth.tsv', sep='\t')


birth = birth_df.values
print(birth[0])

['0 1'
 'Tara Gandhi was born on 24 April 1934 . His mother is Lakshmi Rajagopalachari']


In [28]:
model = archive.model
model.eval()


print("start")
# metrics = evaluate(model, instances, iterator, cuda_device, "")
data_iterator = DataIterator.from_params(Params(
    {"type": "basic", "batch_size": 1}
))
data_iterator.index_with(model.vocab)
iterator = data_iterator(instances,
                            num_epochs=1,
                            shuffle=False)
logger.info("Iterating over dataset")
generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))
rows = 0
with open('birth1.txt', 'wt') as f:
    for instance in generator_tqdm:
        rows+=1

        batch = nn_util.move_to_device(instance, cuda_device)
        output_dict = model(**batch)
        pooled_output = output_dict.get("pooled_output")
        contextual_embeddings = output_dict.get("contextual_embeddings")
        prediction_scores, seq_relationship_score = model.pretraining_heads(
        contextual_embeddings, pooled_output
        )
        prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])
        
        vals, idxs = torch.topk(prediction_scores, k = 5, dim = 1)
        idxs = idxs.cpu().numpy()
        lines = []
        print("row: ", rows, file=f)
        print("================", file=f)
        print("source: ", birth[rows - 1, 1], file = f)
        masked = []
        for id in range(len(idxs)):
            masked += [token2word[instance["tokens"]["tokens"][0][id].item()]]
        masked = " ".join(masked[1: -1])
        print("masked: ", masked, file = f)
        for k in range(5):
            line = []
            for i, idx in enumerate(idxs):
                if instance["tokens"]["tokens"][0][i] != 103:
                    line += [token2word[instance["tokens"]["tokens"][0][i].item()]]
                else:
                    line += [token2word[idx[k]]]
            
            line = " ".join(line[1: -1])
            line = line.split(" ##")
            line = "".join(line)
            print("{}     :  ".format(k) + line, file=f)
        print("================", file=f)
        if rows>99:
            break
    

2022-02-22 17:38:28 - allennlp.common.from_params - INFO: - instantiating class <class 'allennlp.data.iterators.data_iterator.DataIterator'> from params {'type': 'basic', 'batch_size': 1} and extras set()
2022-02-22 17:38:28 - allennlp.common.params - INFO: - type = basic
2022-02-22 17:38:28 - allennlp.common.from_params - INFO: - instantiating class <class 'allennlp.data.iterators.basic_iterator.BasicIterator'> from params {'batch_size': 1} and extras set()
2022-02-22 17:38:28 - allennlp.common.params - INFO: - batch_size = 1
2022-02-22 17:38:28 - allennlp.common.params - INFO: - instances_per_epoch = None
2022-02-22 17:38:28 - allennlp.common.params - INFO: - max_instances_in_memory = None
2022-02-22 17:38:28 - allennlp.common.params - INFO: - cache_instances = False
2022-02-22 17:38:28 - allennlp.common.params - INFO: - track_epoch = False
2022-02-22 17:38:28 - allennlp.common.params - INFO: - maximum_samples_per_batch = None
2022-02-22 17:38:28 - root - INFO: - Iterating over datas

start


  1%|          | 99/11166 [00:05<11:03, 16.69it/s]
